# Очистка данных
Очистка и разведочный анализ - важная часть обучения модели. Очень важно убрать значения которые могут привести к плохим результатам модели. 
В данных датасетах были убраны повторения, убраны слишком короткие слова, а также описания были очищены от знаков препинания. При преобразовании кода ВЭД в число в некоторых кодах была потеряна цифра 0, если с нее начинался код, поэтому ТНВЭД был преобразован в текст и мы вернули все числа на свои места.
Полный код итоговой подготовки данных представлен ниже.

In [ ]:
#  Загрузим библиотеки. Все библиотеки бесплатны и основаны на открытом исходном коде.
import os
import re
import pandas as pd


#  Напишем функции для очистки данных.
def clear_data_ved(df1, save: bool = False, path: str = None, name: str = 'ved.csv'):
    '''docstring
    Функция принимает датафрейм и возвращает очищенный датафрейм, с разделенными кодами ВЭД/
    Функция возвращает датафрейм. Также есть возможность сохранить файл по нужному пути и менять имя для сохранения.
    '''
    df = df1.copy()
    df = df.astype({'KOD_TNVED_SPR': 'str'}) 
    df.KOD_TNVED_SPR = df.KOD_TNVED_SPR.apply(lambda x: '0' + x if len(x) == 9 else x) #  возвращаем 0 в начало если код начинался с 0
    df['KOD_TNVED_FIRST_4'] = df.KOD_TNVED_SPR.str.slice(0, 4) #  колонка с 4 первыми цифрами
    df['KOD_TNVED_LAST_6'] = df.KOD_TNVED_SPR.str.slice(-6) #  остальные 6 цифр кода
    df = df.drop(columns='KOD_TNVED_SPR')
    df = df.drop(columns=df.columns[0], axis=1)
    if save and path is not None:
        path = os.path.join(path, name)
        df.to_csv(path)
    elif save and path is None:
        df.to_csv(name)
    return df


def clear_data_opisanie(df2, save: bool = False, path: str = None, name: str = 'ved.csv'):
    '''docstring
    Функция принимает датафрейм и возвращает очищенный датафрейм, с разделенными кодами ВЭД/
    Функция возвращает датафрейм. Также есть возможность сохранить файл по нужному пути и менять имя для сохранения.
    '''
    df = df2.copy()
    df = df[['TNVED', 'OPISANIE']].drop_duplicates()
    df['len_opisanie'] = list(map(len, df.OPISANIE)) #  получаем длину описаний.
    df = df.query('len_opisanie != 1') #  убираем бессмысленные и пустые ячейки.
    df = df.query('len_opisanie != 2')
    df.reset_index(inplace=True)
    try: #  удаление определенной строки связано с этим датасетом и на другом может привести к ошибке.
        df = df.drop(index=[1363489]) #  при длине в 3 знака здесь были только знаки препинания.
    except:
        pass
    df = df.drop(['index'], axis=1)
    #  ставим регулярку для того чтобы избавиться от знаков препинания.
    pattern = r"""(?:[A-Z]\.)+ |\d+(?:\.\d+)?%?|\w/.+$\s-|\w+(?:[-']\w+)*|
    (?:[+/\-@&*]|/.$/)"""
    df.OPISANIE = df.OPISANIE.apply(lambda x: ' '.join(re.findall(pattern, x))) #  убираем знаки препинания
    df.drop(['len_opisanie'], axis=1, inplace=True)
    df = df.astype({'TNVED': 'str'})
    df.TNVED = df.TNVED.apply(lambda x: '0' + x if len(x) == 3 else x) #  возвращаем 0 в начало кода, если он пропал.
    if save and path is not None:
        path = os.path.join(path, name)
        df.to_csv(path)
    elif save and path is None:
        df.to_csv(name)
    return df

In [ ]:
#  загружаем данные
df1 = pd.read_csv('tnveddata_20211126.csv', encoding='mbcs', sep=';')
df2 = pd.read_csv('dataset_20211126.csv', encoding='mbcs', sep=';')

df1_1 = clear_data_ved(df1, save=True, name='ved1.csv')
df2_1 = clear_data_opisanie(df2, save=True, name='ved2.csv')